<a href="https://colab.research.google.com/github/Ananya-Jha-code/Injury-Classification-CNN/blob/main/Injury_Classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import cv2
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [16]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [17]:
!ls "/content/drive/MyDrive/injury"

test  train  valid


In [18]:
data_dir = '/content/drive/MyDrive/injury'
train_dir = '/content/drive/MyDrive/injury/train'
valid_dir = '/content/drive/MyDrive/injury/valid'
test_dir = '/content/drive/MyDrive/injury/test'

In [50]:
# define transforms for training, validation and test sets
training_transforms = transforms.Compose([transforms.RandomRotation(30),
                                         transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485,0.456,0.406],
                                                               [0.229,0.224,0.225])])
validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485,0.456,0.406],
                                                                 [0.229,0.224,0.225])])
testing_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485,0.456,0.406],
                                                           [0.229,0.224,0.225])])

In [21]:
#load datasets with ImageFolder
training_dataset = datasets.ImageFolder(train_dir, transform=training_transforms)
validation_dataset = datasets.ImageFolder(valid_dir, transform=validation_transforms)
testing_dataset = datasets.ImageFolder(test_dir, transform=testing_transforms)

In [22]:
# define dataloaders
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)
validate_loader = torch.utils.data.DataLoader(validation_dataset,batch_size=32)
test_loader = torch.utils.data.DataLoader(testing_dataset,batch_size=32)

In [31]:
def categories(img,injury_type):
  return injury_type
def make_train_data(injury_type,location):
  for img in tqdm(os.listdir(location)):
    label = categories(img,injury_type)
    path = os.path.join(location,img)
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))

    X.append(np.array(img))
    y.append(str(label))

In [5]:
#build and train network
#transfer learning
from torchvision import models
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [33]:
# no backpropagation on pretrained set to save time
for parameter in model.parameters():
    parameter.requires_grad = False


from collections import OrderedDict

# Build custom classifier
from torch import nn
classifier = nn.Sequential(OrderedDict([('fc1',nn.Linear(25088,5000)),
                                        ('relu',nn.ReLU()),
                                        ('drop',nn.Dropout(p=0.5)),#to avoid overfitting
                                        ('fc2',nn.Linear(5000,102)),
                                        ('output',nn.LogSoftmax(dim=1))]))

In [68]:
#Function for validation pass
def validation (model, validateloader, criterion):
  val_loss = 0
  accuracy = 0
  for images, labels in iter(validateloader):
    images, labels = images.to('cuda'), labels.to('cuda')
    output = model.forward(images)
    val_loss += criterion(output, labels).item()

    probabilities = torch.exp(output)
    equality = (labels.data == probabilities.max(dim=1)[1])
    accuracy += equality.type(torch.FloatTensor).mean()
    return val_loss, accuracy

In [24]:
#loss function and gradient descent
from torch import optim
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [36]:
import torch
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [2]:
pip install workspace

  Created wheel for workspace: filename=workspace-0.3.1-cp37-none-any.whl size=6188 sha256=121c4e2781f40c3d93f5308b729674c3356e2f1c6c032211cd0a92286ae04cc7
  Stored in directory: /root/.cache/pip/wheels/47/ad/78/a8b0e4959544289f25fca64d414165290ba285837b38c07441
  Created wheel for sprinkles: filename=sprinkles-0.4.6-cp37-none-any.whl size=4995 sha256=e8449fba659e5bd6f8e3c3deb47238b476d0d16934271bbf1cf29d5aa5590a23
  Stored in directory: /root/.cache/pip/wheels/2d/09/fa/e5a0f4b5064e5b6df5f2294039dcf0f414ef82a4a8c66c8d69
Successfully built workspace sprinkles


In [3]:
pip install utils

In [69]:
# Train the classifier


def train_classifier():


        epochs = 15
        steps = 0
        print_every = 40

        model.to('cuda')

        for e in range(epochs):
        
            model.train()
    
            running_loss = 0
    
            for images, labels in iter(train_loader):
        
                steps += 1
        
                images, labels = images.to('cuda'), labels.to('cuda')
        
                optimizer.zero_grad()

                output = model.forward(images)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()
        
                running_loss += loss.item()
        
                if steps % print_every == 0:
                
                    model.eval()
                
                    # Turn off gradients for validation, saves memory and computations
                    with torch.no_grad():
                      validation_loss, accuracy = validation(model, validate_loader, criterion)
            
                    print("Epoch: {}/{}.. ".format(e+1, epochs),
                          "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                          "Validation Loss: {:.3f}.. ".format(validation_loss/len(validate_loader)),
                          "Validation Accuracy: {:.3f}".format(accuracy/len(validate_loader)))
            
                    running_loss = 0
                    model.train()
                    
train_classifier()

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: 2/15..  Training Loss: -773268275.200..  Validation Loss: -1235238707.200..  Validation Accuracy: 0.100
Epoch: 3/15..  Training Loss: -1852736409.600..  Validation Loss: -1561016422.400..  Validation Accuracy: 0.100
Epoch: 4/15..  Training Loss: -3656838144.000..  Validation Loss: -1936007782.400..  Validation Accuracy: 0.100
Epoch: 5/15..  Training Loss: -5658140262.400..  Validation Loss: -2362609254.400..  Validation Accuracy: 0.100
Epoch: 6/15..  Training Loss: -8821705164.800..  Validation Loss: -2842926080.000..  Validation Accuracy: 0.100
Epoch: 7/15..  Training Loss: -12044110336.000..  Validation Loss: -3376920371.200..  Validation Accuracy: 0.100
Epoch: 8/15..  Training Loss: -16860113612.800..  Validation Loss: -3978799923.200..  Validation Accuracy: 0.100
Epoch: 9/15..  Training Loss: -22202621747.200..  Validation Loss: -4642648064.000..  Validation Accuracy: 0.100
Epoch: 10/15..  Training Loss: -29074581401.600..  Validation Loss: -5364632371.200..  Validation Accu

In [70]:
def test_accuracy(model, test_loader):

    # Do validation on the test set
    model.eval()
    model.to('cuda')

    with torch.no_grad():
    
        accuracy = 0
    
        for images, labels in iter(test_loader):
    
            images, labels = images.to('cuda'), labels.to('cuda')
    
            output = model.forward(images)

            probabilities = torch.exp(output)
        
            equality = (labels.data == probabilities.max(dim=1)[1])
        
            accuracy += equality.type(torch.FloatTensor).mean()
        
        print("Test Accuracy: {}".format(accuracy/len(test_loader)))    
        
        
test_accuracy(model, test_loader)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Test Accuracy: 0.3125
